In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Bang Len,TH,14.02,100.17,86.00,79,40,8.05,scattered clouds,0.0,0.0
1,1,Bluff,NZ,-46.60,168.33,48.52,78,85,8.59,overcast clouds,0.0,0.0
2,2,Nanortalik,GL,60.14,-45.24,43.75,79,92,10.98,overcast clouds,0.0,0.0
3,3,Ushuaia,AR,-54.80,-68.30,37.40,74,75,14.99,broken clouds,0.0,0.0
4,4,Jamestown,US,42.10,-79.24,73.40,60,75,12.75,broken clouds,0.0,0.0


In [3]:
# Ask the customer to add a minimum and maximum temperature value and snowing/rainign prefs.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
raining = str(input("Do you want it to be raining? (yes/no) "))
snowing = str(input("Do you want it to be snowing? (yes/no) "))
snowing_bool = (snowing.lower() == "yes")
raining_bool = (raining.lower() == "yes")

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no) No
Do you want it to be snowing? (yes/no) No


In [4]:
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
if raining_bool:
    preferred_cities_df = preferred_cities_df.loc[preferred_cities_df["Rain (inches)"] > 0.0]
else:
    preferred_cities_df = preferred_cities_df.loc[preferred_cities_df["Rain (inches)"] == 0.0]

if snowing_bool:
    preferred_cities_df = preferred_cities_df.loc[preferred_cities_df["Snow (inches)"] > 0.0]
else:
    preferred_cities_df = preferred_cities_df.loc[preferred_cities_df["Snow (inches)"] == 0.0]
    
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Bang Len,TH,14.02,100.17,86.00,79,40,8.05,scattered clouds,0.0,0.0
15,15,Dingle,PH,11.00,122.67,76.41,87,100,2.62,overcast clouds,0.0,0.0
31,31,Atuona,PF,-9.80,-139.03,80.02,77,43,14.81,scattered clouds,0.0,0.0
36,36,Oktyabrskoye,RU,43.06,44.74,75.20,50,10,4.47,clear sky,0.0,0.0
39,39,Kapaa,US,22.08,-159.32,77.00,83,90,9.17,overcast clouds,0.0,0.0


In [5]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, description, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bang Len,TH,86.00,scattered clouds,14.02,100.17,
15,Dingle,PH,76.41,overcast clouds,11.00,122.67,
31,Atuona,PF,80.02,scattered clouds,-9.80,-139.03,
36,Oktyabrskoye,RU,75.20,clear sky,43.06,44.74,
39,Kapaa,US,77.00,overcast clouds,22.08,-159.32,
41,Vaini,TO,75.20,scattered clouds,-21.20,-175.20,
43,Georgetown,MY,84.20,few clouds,5.41,100.34,
48,Palmer,US,86.00,broken clouds,42.16,-72.33,
50,Luoyang,CN,75.00,broken clouds,34.68,112.45,
55,Faya,SA,84.20,few clouds,18.39,42.45,


In [6]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [7]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bang Len,TH,86.00,scattered clouds,14.02,100.17,บ้านสอนพิเศษอังกฤษ คณิต
15,Dingle,PH,76.41,overcast clouds,11.00,122.67,Camp Pasica
31,Atuona,PF,80.02,scattered clouds,-9.80,-139.03,Villa Enata
36,Oktyabrskoye,RU,75.20,clear sky,43.06,44.74,Prolesok Mini-hotel
39,Kapaa,US,77.00,overcast clouds,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
41,Vaini,TO,75.20,scattered clouds,-21.20,-175.20,Keleti Beach Resort
43,Georgetown,MY,84.20,few clouds,5.41,100.34,Cititel Penang
48,Palmer,US,86.00,broken clouds,42.16,-72.33,Wedgewood Motel
50,Luoyang,CN,75.00,broken clouds,34.68,112.45,易家国际青年旅舍
55,Faya,SA,84.20,few clouds,18.39,42.45,مقبرة ال مجمل


In [8]:
# Create the output file (CSV).
output_data_file = "data/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [9]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [15]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)
# Add a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))